# Statewide Data with Crash Severity as Target

## Imports and Setup

In [1]:
# Initial imports
import pandas as pd
import numpy as np
import psycopg2 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Load in password
from secret import secret, database, username, host

# Establish connection to database
conn = psycopg2.connect(
    host = host,
    database = database,
    user =username,
    password = secret
)

In [3]:
# Load the data
query = 'SELECT * FROM clark_co_traffic'
traffic_df = pd.read_sql_query(query, conn)

/var/folders/f3/5zcd9_1j7d3546ws7g9t5fw00000gp/T/ipykernel_6199/3027837082.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  traffic_df = pd.read_sql_query(query, conn)


## Preprocess Data

In [4]:
# Drop unnecessary columns
traffic_df.drop(columns = ['objectid','injury_type','accident_rec_num','primary_street', 'secondary_street', 'fatalities', 'injured','property_damage_only' ], inplace = True)

In [5]:
traffic_df.columns

Index(['x', 'y', 'crash_severity', 'county', 'crash_date', 'crash_year',
       'crash_time', 'weather', 'crash_type', 'total_vehicles', 'v1_type',
       'v1_driver_age', 'v1_action', 'v1_driver_factors',
       'v1_driver_distracted', 'v1_vehicle_factors', 'v1_most_harmful_event',
       'v1_all_events', 'v2_type', 'v2_driver_age', 'v2_action',
       'v2_driver_factors', 'v2_driver_distracted', 'v2_vehicle_factors',
       'v2_most_harmful_event', 'v2_all_events', 'nonmotorist_factors',
       'factors_roadway', 'lighting', 'hwy_factors', 'pedalcyclist',
       'pedestrian', 'motorcyclist', 'animaltype'],
      dtype='object')

In [6]:
# Split single column with multiple data points into separate columns
traffic_df[['v1_driver_factors_1', 'v1_driver_factors_2','v1_driver_factors_3', 'v1_driver_factors_4']] = traffic_df['v1_driver_factors'].str.split(':', expand=True)

traffic_df[['v1_vehicle_factors_1', 'v1_vehicle_factors_2','v1_vehicle_factors_3', 'v1_vehicle_factors_4', 'v1_vehicle_factors_5']] = traffic_df['v1_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v1_all_events_1', 'v1_all_events_2','v1_all_events_3', 'v1_all_events_4', 'v1_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['v2_driver_factors_1', 'v2_driver_factors_2']] = traffic_df['v2_driver_factors'].str.split(':', expand=True)

traffic_df[['v2_vehicle_factors_1', 'v2_vehicle_factors_2','v2_vehicle_factors_3', 'v2_vehicle_factors_4', 'v2_vehicle_factors_5']] = traffic_df['v2_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v2_all_events_1', 'v2_all_events_2','v2_all_events_3', 'v2_all_events_4', 'v2_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['nonmotorist_factors_1', 'nonmotorist_factors_2','nonmotorist_factors_3', 'nonmotorist_factors_4', 'nonmotorist_factors_5']] = traffic_df['nonmotorist_factors'].str.split(':', expand=True)

traffic_df[['factors_roadway_1', 'factors_roadway_2','factors_roadways_3']] = traffic_df['factors_roadway'].str.split(':', expand=True)

traffic_df[['hwy_factors_1', 'hwy_factors_2','hwy_factors_3', 'hwy_factors_4']] = traffic_df['hwy_factors'].str.split(':', expand=True)

traffic_df.drop(columns = ['v1_driver_factors','v1_vehicle_factors','v1_all_events','v2_driver_factors','v2_vehicle_factors', 'v2_all_events', 'nonmotorist_factors', 'factors_roadway', 'hwy_factors',], inplace = True)

In [7]:
traffic_df['crash_severity'].value_counts()

PROPERTY DAMAGE ONLY    128261
INJURY CRASH             96232
FATAL CRASH               1175
Name: crash_severity, dtype: int64

In [8]:
# Change target values to numeric
# Values are based on Nevada Traffic Records Coordinating Committee Data Dictionary,
# https://zerofatalitiesnv.com/app/uploads/2021/04/2021-01-NV-TRCC-Data-Dictionary.pdf
injury_num = {
    'PROPERTY DAMAGE ONLY': 0,
    'INJURY CRASH': 1,
    'FATAL CRASH': 2,    
}

traffic_df['crash_severity'] = traffic_df['crash_severity'].apply(lambda x: injury_num[x])

traffic_df['crash_severity'].value_counts()

0    128261
1     96232
2      1175
Name: crash_severity, dtype: int64

In [9]:
# Extract datetime values into separate columns
traffic_df['crash_month'] = pd.DatetimeIndex(traffic_df['crash_date']).month
traffic_df['crash_day'] = pd.DatetimeIndex(traffic_df['crash_date']).day
traffic_df['crash_day_of_week'] = pd.DatetimeIndex(traffic_df['crash_date']).dayofweek
traffic_df['crash_hour'] = pd.DatetimeIndex(traffic_df['crash_time']).hour


In [10]:
# Drop original datetime columns
traffic_df.drop(columns = ['crash_date', 'crash_time'], inplace = True)

In [11]:
traffic_df.dtypes

x                    float64
y                    float64
crash_severity         int64
county                object
crash_year             int64
                      ...   
hwy_factors_4         object
crash_month            int64
crash_day              int64
crash_day_of_week      int64
crash_hour             int64
Length: 65, dtype: object

In [12]:
# Get names of all columns with dtypes of "object"
sel_cols = list(traffic_df.select_dtypes(include='object'))

In [13]:
# Preprocessing to convert all strings to numeric values
traffic_df_encoded = pd.get_dummies(traffic_df, columns = sel_cols)
traffic_df_encoded.head()

,x,y,crash_severity,crash_year,total_vehicles,v1_driver_age,v2_driver_age,crash_month,crash_day,crash_day_of_week,...,hwy_factors_3_ OTHER HIGHWAY,hwy_factors_3_ ROAD OBSTRUCTION,"hwy_factors_3_ RUTS, HOLES, BUMPS",hwy_factors_3_ SHOULDERS,hwy_factors_3_ VISUAL OBSTRUCTION(S),hwy_factors_3_ WEATHER,"hwy_factors_3_ WET, ICY, SNOW, SLUSH",hwy_factors_3_ WORK ZONE (CONST. MAINT. UTILITY),hwy_factors_3_ WORN TRAFFIC SURFACE,"hwy_factors_4_ WET, ICY, SNOW, SLUSH"
0,-115.106709,36.236043,0,2016,2,35,41,1,4,0,...,0,0,0,0,0,0,0,0,0,0
1,-119.673794,39.626433,0,2016,1,39,41,1,7,3,...,0,0,0,0,0,0,0,0,0,0
2,-115.101063,36.240200,0,2016,2,31,53,1,4,0,...,0,0,0,0,0,0,0,0,0,0
3,-119.633092,39.516952,0,2016,1,18,41,1,6,2,...,0,0,0,0,0,0,0,0,0,0
4,-115.140570,36.192680,0,2016,1,56,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0


## Model with Crash Severity as Target and All Other Columns as Features

In [14]:
# Separate data into features and target
y = traffic_df_encoded['crash_severity'].values
X = traffic_df_encoded.drop(columns = 'crash_severity')

In [15]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)
X_train.shape

(169251, 1345)

In [16]:
# Scale the model
scaler = StandardScaler() 

# Fit the scaler with the training data
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Create and fit the Random Forest model
rf_model = RandomForestClassifier(n_estimators = 128)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [18]:
# Create predictions
predictions = rf_model.predict(X_test_scaled)
results = pd.DataFrame({
    "Prediction": predictions,
    "Actual": y_test
}).reset_index(drop = True)
results.head()

,Prediction,Actual
0,1,1
1,1,1
2,0,0
3,0,1
4,1,1


In [19]:
# Assess accuracy score
accuracy = accuracy_score(y_test, predictions)
accuracy

0.7342290444369605

In [20]:
# Generate confusion matrix and dataframe
cm = confusion_matrix(y_test, predictions)

index_values = ['Actual: Property Damage Only', 
                'Actual: Injury Crash',  
                'Actual: Fatal Crash']

column_values = ['Predicted: Property Damage Only', 
                'Predicted: Injury Crash', 
                'Predicted: Fatal Crash']

cm_df = pd.DataFrame(cm, index = index_values, columns = column_values)
cm_df
                  

,Predicted: Property Damage Only,Predicted: Injury Crash,Predicted: Fatal Crash
Actual: Property Damage Only,25141,6923,1
Actual: Injury Crash,7781,16270,7
Actual: Fatal Crash,70,212,12


## Statewide Results with Crash Severity as Target and All Other Columns as Features

In [21]:
# Display results
print('Confusion Matrix')
display(cm_df)
print(f'\nAccuracy Score: {accuracy}\n')
print('Classification Report')
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted: Property Damage Only,Predicted: Injury Crash,Predicted: Fatal Crash
Actual: Property Damage Only,25141,6923,1
Actual: Injury Crash,7781,16270,7
Actual: Fatal Crash,70,212,12



Accuracy Score: 0.7342290444369605

Classification Report
              precision    recall  f1-score   support

           0       0.76      0.78      0.77     32065
           1       0.70      0.68      0.69     24058
           2       0.60      0.04      0.08       294

    accuracy                           0.73     56417
   macro avg       0.69      0.50      0.51     56417
weighted avg       0.73      0.73      0.73     56417



### Importances with Crash Severity as Target and All Other Columns as Features

In [22]:
# Calculate importance of features
importances = rf_model.feature_importances_
listo = sorted(zip(rf_model.feature_importances_, X.columns), reverse = True)
listo

[(0.051701003284283814, 'y'),
 (0.04991091282371679, 'x'),
 (0.04842625326046089, 'v1_driver_age'),
 (0.0431550305693124, 'v2_driver_age'),
 (0.0412534033599841, 'crash_day'),
 (0.038220804348866225, 'crash_hour'),
 (0.03471637268565583, 'crash_month'),
 (0.030273613332567954, 'crash_day_of_week'),
 (0.024260904642841533, 'lighting_UNKNOWN'),
 (0.022958875379670486, 'factors_roadway_1_UNKNOWN'),
 (0.022701067006609517, 'hwy_factors_1_UNKNOWN'),
 (0.020402321698287443, 'crash_year'),
 (0.01987010861608564, 'hwy_factors_1_NONE'),
 (0.018038847943717514, 'factors_roadway_1_DRY'),
 (0.014035809085526698, 'total_vehicles'),
 (0.012064697137613985, 'lighting_DAYLIGHT'),
 (0.009942430119695905, 'v1_type_SEDAN, 4 DOOR'),
 (0.00940448580534137, 'v1_vehicle_factors_1_FAILED TO YIELD RIGHT OF WAY'),
 (0.008982348815487685, 'crash_type_SIDESWIPE, OVERTAKING'),
 (0.008701577905088008, 'crash_type_ANGLE'),
 (0.008054717310951767, 'v1_action_GOING STRAIGHT'),
 (0.007741778057865253, 'v2_type_SEDAN, 4

## New Version of Model Based on Top Importances

### Importances to Exclude

In [23]:
important = sorted(zip(rf_model.feature_importances_, X.columns), reverse = True)
low_priority = important[772:]

In [24]:
imp_only_list = []
listy_list = []

# iterate using index with enumerate function
for index, tuple in enumerate(low_priority):
       
    # access through index
      # by appending to list
    imp_only_list.append(low_priority[index])
     
# iterate through the list
for x in imp_only_list:
    for y in x:
         listy_list.append(y)

final_list = listy_list[1::2]


### Dataframe without Low-Value Importances

In [25]:
important_df = traffic_df_encoded.drop(columns = final_list)
important_df.head()

,x,y,crash_severity,crash_year,total_vehicles,v1_driver_age,v2_driver_age,crash_month,crash_day,crash_day_of_week,...,hwy_factors_2_ NON-ROADWAY WORK,hwy_factors_2_ OTHER ENVIRONMENTAL,hwy_factors_2_ OTHER HIGHWAY,"hwy_factors_2_ WET, ICY, SNOW, SLUSH",hwy_factors_2_ WORK ZONE (CONST. MAINT. UTILITY),hwy_factors_3_ BACKUP DUE TO PRIOR CRASH,hwy_factors_3_ BACKUP DUE TO REGULAR CONGESTION,hwy_factors_3_ OTHER HIGHWAY,"hwy_factors_3_ WET, ICY, SNOW, SLUSH",hwy_factors_3_ WORK ZONE (CONST. MAINT. UTILITY)
0,-115.106709,36.236043,0,2016,2,35,41,1,4,0,...,0,0,0,0,0,0,0,0,0,0
1,-119.673794,39.626433,0,2016,1,39,41,1,7,3,...,0,0,0,0,0,0,0,0,0,0
2,-115.101063,36.240200,0,2016,2,31,53,1,4,0,...,0,0,0,0,0,0,0,0,0,0
3,-119.633092,39.516952,0,2016,1,18,41,1,6,2,...,0,0,0,0,0,0,0,0,0,0
4,-115.140570,36.192680,0,2016,1,56,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0


## Model with Crash Severity as Target and Only High-Value Importances as Features

In [26]:
# Separate data into features and target
y_imp = important_df['crash_severity'].values

X_imp = important_df.drop(columns = 'crash_severity')

In [27]:
# had to remove stratify because there was a singleton array
X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(X_imp, y_imp)
X_train_imp.shape

(169251, 772)

In [28]:
# Scale the model
scaler_imp = StandardScaler() 

# Fit the scaler with the training data
X_scaler_imp = scaler_imp.fit(X_train_imp)
X_train_scaled_imp = X_scaler_imp.transform(X_train_imp)
X_test_scaled_imp = X_scaler_imp.transform(X_test_imp)

In [29]:
# Create and fit the Random Forest model
rf_model_imp = RandomForestClassifier(n_estimators = 128)
rf_model_imp = rf_model.fit(X_train_scaled_imp, y_train_imp)

In [30]:
# Create predictions
predictions_imp = rf_model_imp.predict(X_test_scaled_imp)
results_imp = pd.DataFrame({
    "Prediction": predictions_imp,
    "Actual": y_test_imp
}).reset_index(drop = True)
results_imp.head()

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,1,0
4,0,1


In [31]:
# Assess accuracy score
accuracy_imp = accuracy_score(y_test_imp, predictions_imp)
accuracy_imp

0.7400074445645816

In [32]:
# Generate confusion matrix and dataframe
cm_imp = confusion_matrix(y_test_imp, predictions_imp)

index_values_imp = ['Actual: Property Damage Only', 
                'Actual: Injury Crash',  
                'Actual: Fatal Crash']

column_values_imp = ['Predicted: Property Damage Only', 
                'Predicted: Injury Crash', 
                'Predicted: Fatal Crash']

cm_df_imp = pd.DataFrame(cm_imp, index = index_values_imp, columns = column_values_imp)
cm_df_imp

,Predicted: Property Damage Only,Predicted: Injury Crash,Predicted: Fatal Crash
Actual: Property Damage Only,25380,6737,1
Actual: Injury Crash,7675,16354,5
Actual: Fatal Crash,60,190,15


## Statewide Results with Crash Severity as Target and High-Priority Columns as Features

In [33]:
# Display results
print('Confusion Matrix')
display(cm_df_imp)
print(f'\nAccuracy Score: {accuracy_imp}\n')
print('Classification Report')
print(classification_report(y_test_imp, predictions_imp))

Confusion Matrix


,Predicted: Property Damage Only,Predicted: Injury Crash,Predicted: Fatal Crash
Actual: Property Damage Only,25380,6737,1
Actual: Injury Crash,7675,16354,5
Actual: Fatal Crash,60,190,15



Accuracy Score: 0.7400074445645816

Classification Report
              precision    recall  f1-score   support

           0       0.77      0.79      0.78     32118
           1       0.70      0.68      0.69     24034
           2       0.71      0.06      0.10       265

    accuracy                           0.74     56417
   macro avg       0.73      0.51      0.52     56417
weighted avg       0.74      0.74      0.74     56417



### Calculate Importance of Features

In [34]:
importances_high_priority = rf_model_imp.feature_importances_
sorted(zip(importances_high_priority, X_imp.columns), reverse = True)

[(0.0519294170277878, 'y'),
 (0.05154978454448769, 'x'),
 (0.049980679009283394, 'v1_driver_age'),
 (0.044790169160743014, 'v2_driver_age'),
 (0.04140547947780209, 'crash_day'),
 (0.038520610379419305, 'crash_hour'),
 (0.03466621427708862, 'crash_month'),
 (0.02997998278139124, 'crash_day_of_week'),
 (0.0292156437305588, 'factors_roadway_1_UNKNOWN'),
 (0.02345550545219998, 'lighting_UNKNOWN'),
 (0.019820666087610073, 'crash_year'),
 (0.019383827276353217, 'hwy_factors_1_UNKNOWN'),
 (0.016939403847081563, 'hwy_factors_1_NONE'),
 (0.016321506347591358, 'factors_roadway_1_DRY'),
 (0.013808718849559769, 'total_vehicles'),
 (0.012647168826224074, 'lighting_DAYLIGHT'),
 (0.010087109024687221, 'crash_type_SIDESWIPE, OVERTAKING'),
 (0.010043748372842649, 'v1_vehicle_factors_1_FAILED TO YIELD RIGHT OF WAY'),
 (0.009851054089503807, 'v1_type_SEDAN, 4 DOOR'),
 (0.00801290682733818, 'v1_action_GOING STRAIGHT'),
 (0.007973845884355837, 'crash_type_ANGLE'),
 (0.007623757486635064, 'v2_type_SEDAN, 4 